<a href="https://colab.research.google.com/github/MCV-Jimenez/Food-sales-predictions/blob/main/Project_1_Part_2_MJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Food Sales Predictions**

Marco Jimenez

2/6/2022

In [258]:
import pandas as pd

In [259]:
import numpy as np

In [260]:
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

In [261]:
sales_predictions = '/content/sales_predictions.csv'
df_sales = pd.read_csv(sales_predictions)
df_sales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [262]:
#1 Identifying the number of rows and columns
rows = df_sales.shape[0]
cols = df_sales.shape[1]
# Displaying the number of rows and columns
print("Rows:" + str(rows))
print("Columns:" + str(cols))

Rows:8523
Columns:12


In [263]:
#2 Identifying data types
df_sales.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [264]:
#3 Removing any duplicated rows 
df_sales.drop_duplicates()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [265]:
#4 Identifying missing values
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [266]:
#5 Filling in the missing values in the Item Weight column by mean
# I refered to a link posted by Mandy in the discord and used the structure of the code she suggested. From what I understand it's grouping the missing weight values by their respective Item Identifiers & filling them in with the average item weight of their group  
df['Item_Weight'] = df_sales['Item_Weight'].fillna(df_sales.groupby('Item_Identifier')['Item_Weight'].transform('mean'))
# For some reason there are still four missing values but I was unbale to determine why or how to fill them in even with the fillna method
df['Item_Weight'].isnull().sum()

4

In [267]:
#5 Filled the NaN values in the Outlet Size column with "missing" because dropping those rows would remove too much important information and there was no way to group it with another column in a way that made sense relative to the size of any given store.
df['Outlet_Size'] = df_sales[['Outlet_Size']].fillna("missing")
df['Outlet_Size'] 

0        Medium
1        Medium
2        Medium
3       missing
4          High
         ...   
8518       High
8519    missing
8520      Small
8521     Medium
8522      Small
Name: Outlet_Size, Length: 8523, dtype: object

In [268]:
#6 Confirming that there are no more missing values
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [269]:
#7 Identifying how many unique values are in each column
df_sales.nunique()

Item_Identifier              1559
Item_Weight                   415
Item_Fat_Content                5
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

In [270]:
#7 Looking at insconsistencies in the Item Fat Content column
df_sales['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [271]:
# Creating a dictionary of the values I want to replace and what I want to replace them with
replace_fat = {'reg':"Regular",'low fat':"Low Fat", 'LF':"Low Fat"}

In [272]:
#7 The Item Fat Content column should only have the 2 values 'Low Fat' or 'Regular'
# For this answer I referenced a student/instructor exchange on this question in the discord. 
# Filtering the unique values in the fat content column
Item_Fat_Content = df['Item_Fat_Content'].unique()
# Creating a for loop function for the column to replace the incorrect values with the correct values
for Item_Fat_Content in df_sales.Item_Fat_Content:     
    if Item_Fat_Content == "reg":
        print("Regular")
    elif Item_Fat_Content == "low fat":
        print("Low Fat")
    elif Item_Fat_Content == 'LF':
        print("Low Fat")
        continue
        print(Item_Fat_Content)

Low Fat
Low Fat
Low Fat
Low Fat
Regular
Low Fat
Low Fat
Low Fat
Low Fat
Regular
Low Fat
Regular
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Regular
Low Fat
Regular
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Regular
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Regular
Low Fat
Low Fat
Regular
Regular
Low Fat
Low Fat
Low Fat
Regular
Low Fat
Low Fat
Low Fat
Regular
Low Fat
Low Fat
Low Fat
Low Fat
Regular
Low Fat
Regular
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Regular
Low Fat
Low Fat
Low Fat
Regular
Regular
Low Fat
Low Fat
Regular
Regular
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Regular
Low Fat
Low Fat
Low Fat
Low Fat
Regular
Low Fat
Low Fat
Regular
Low Fat
Regular
Regular
Low Fat
Regular
Low Fat
Low Fat
Regular
Regular
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Regular
Low Fat
Low Fat
Regular
Low Fat
Regular
Regular
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat
Low Fat


In [273]:
# Applying the for loop function and confirming desired result
df.Item_Fat_Content = df_sales.Item_Fat_Content.replace(replace_fat)
df.Item_Fat_Content.value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [274]:
#7 Based on the consistency of the Outlet Type column I can assume that Grocey Store is not a proper entry for this column
df['Outlet_Type'].value_counts()

Supermarket Type1    5577
missing              1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [275]:
#7 Since I can't relate 'grocery store' to any particular size I am changing it to 'missing' to provide clarity as to wether or not we know what type of supermarket it is 
df['Outlet_Type'] = df['Outlet_Type'].str.replace('Grocery Store','missing')
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,missing,Tier 3,missing,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,missing,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [276]:
#8 Selecting only the numerical columns
mmm_df = df.describe(include=[np.number])
mmm_df

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,8519.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.875420,0.066132,140.992782,1997.831867,2181.288914
std,4.646098,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.785000,0.026989,93.826500,1987.000000,834.247400
50%,12.650000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [277]:
#8 Displaying the min of each numerical column 
mmm_df.min()

Item_Weight                   4.55500
Item_Visibility               0.00000
Item_MRP                     31.29000
Outlet_Establishment_Year     8.37176
Item_Outlet_Sales            33.29000
dtype: float64

In [278]:
#8 Displaying the max of each numerical column
mmm_df.max()

Item_Weight                   8519.0000
Item_Visibility               8523.0000
Item_MRP                      8523.0000
Outlet_Establishment_Year     8523.0000
Item_Outlet_Sales            13086.9648
dtype: float64

In [279]:
#8 Dispplaying the mean of each numerical column
mmm_df.mean()

Item_Weight                  1075.088940
Item_Visibility              1065.452703
Item_MRP                     1180.866156
Outlet_Establishment_Year    2564.150453
Item_Outlet_Sales            3907.614766
dtype: float64